In [39]:
from transformers import (
    ASTForAudioClassification,
    ASTFeatureExtractor,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset, Audio
import numpy as np
import evaluate

In [34]:
gtzan = load_dataset("marsyas/gtzan", "all", trust_remote_code=True)
print(gtzan)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})


In [16]:
# Create test/train split
gtzan_with_train_test_split = gtzan["train"].train_test_split(
    seed=42, shuffle=True, test_size=0.1
)
print(gtzan_with_train_test_split)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})


In [29]:
model_id = "MIT/ast-finetuned-audioset-10-10-0.4593"

feature_extractor = ASTFeatureExtractor.from_pretrained(model_id)

model = ASTForAudioClassification.from_pretrained(model_id, output_attentions=True)

SAMPLING_RATE = 16000

# cast audio to correct sampling rate
gtzan_with_correct_sampling_rate = gtzan_with_train_test_split.cast_column(
    "audio", Audio(sampling_rate=SAMPLING_RATE)
)

/home/jacob/.cache/pypoetry/virtualenvs/ast-fine-tuned-6uKXIvoX-py3.10/lib/python3.10/site-packages/transformers/audio_utils.py:297: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


In [22]:
# Test feature extractor on first sample
sample = gtzan_with_correct_sampling_rate["train"][0]["audio"]

print(
    f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}, Std Dev: {np.std(sample['array']):.3}"
)

inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}, Std Dev: {np.std(inputs['input_values']):.3}"
)

Mean: 0.000185, Variance: 0.0493, Std Dev: 0.222
inputs keys: ['input_values']
Mean: 0.305, Variance: 0.124, Std Dev: 0.353


In [23]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
gtzan_encoded = gtzan_with_correct_sampling_rate.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values'],
        num_rows: 100
    })
})

In [25]:
# Rename "genre" to "label" for Trainer
gtzan_with_label_col = gtzan_encoded.rename_column("genre", "label")

# Create id2label and label2id for use later
id2label_fn = gtzan_encoded["train"].features["genre"].int2str

id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_with_label_col["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
num_labels = len(id2label)

model = ASTForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10
output_dir = f"{model_name}-finetuned-gtzan"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    report_to="none",
)

In [37]:
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_with_label_col["train"],
    eval_dataset=gtzan_with_label_col["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()